In [144]:
import numpy as np 
import pandas as pd 
from scipy import sparse
from lightfm import LightFM
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import csr_matrix 
from scipy.sparse import coo_matrix 
import time
from sklearn import preprocessing
from lightfm.evaluation import auc_score
from lightfm.evaluation import precision_at_k

In [155]:
movies = pd.read_csv('https://github.com/aoyu2021/RS/blob/main/movie.csv')

In [125]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [126]:
rating_df = pd.read_csv('https://github.com/aoyu2021/RS/blob/main/rating.csv')
rating_df = ratings[:800]

In [127]:
rating_df['userId'] = rating_df['userId'].astype('int')
rating_df['movieId'] = rating_df['movieId'].astype('int')
rating_df.head()

C:\Users\87960\AppData\Local\Temp\ipykernel_16484\3902439210.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rating_df['userId'] = rating_df['userId'].astype('int')
C:\Users\87960\AppData\Local\Temp\ipykernel_16484\3902439210.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rating_df['movieId'] = rating_df['movieId'].astype('int')


,userId,movieId,rating,timestamp
0,1,2,3.5,2005-04-02 23:53:47
1,1,29,3.5,2005-04-02 23:31:16
2,1,32,3.5,2005-04-02 23:33:39
3,1,47,3.5,2005-04-02 23:32:07
4,1,50,3.5,2005-04-02 23:29:40


In [128]:
interactions = rating_df.groupby(['userId', 'movieId'])['rating'].sum().unstack().reset_index().fillna(0).set_index('userId')

In [129]:
interactions.head()

movieId,1,2,3,6,7,10,11,15,16,17,...,7454,7482,7757,8368,8482,8507,8636,8690,8961,31696
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,3.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4.0,3.0,4.0,4.0,3.5,5.0,4.5,3.5,4.0,4.0
2,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,3.0,0.0,4.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,3.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [149]:
def informed_train_test(rating_df, train_ratio):
    split_cut = np.int(np.round(rating_df.shape[0] * train_ratio))
    train_df = rating_df.iloc[0:split_cut]
    test_df = rating_df.iloc[split_cut::]
    test_df = test_df[(test_df['userId'].isin(train_df['userId'])) & (test_df['movieId'].isin(train_df['movieId']))]
    id_cols = ['userId', 'movieId']
    trans_cat_train = dict()
    trans_cat_test = dict()
    for k in id_cols:
        cate_enc = preprocessing.LabelEncoder()
        trans_cat_train[k] = cate_enc.fit_transform(train_df[k].values)
        trans_cat_test[k] = cate_enc.transform(test_df[k].values)

    # --- Encode ratings:
    cate_enc = preprocessing.LabelEncoder()
    ratings = dict()
    ratings['train'] = cate_enc.fit_transform(train_df.rating)
    ratings['test'] = cate_enc.transform(test_df.rating)

    n_users = len(np.unique(trans_cat_train['userId']))
    n_items = len(np.unique(trans_cat_train['movieId']))


    train = coo_matrix((ratings['train'], (trans_cat_train['userId'], \
                                                          trans_cat_train['movieId'])) \
                                      , shape=(n_users, n_items))
    test = coo_matrix((ratings['test'], (trans_cat_test['userId'], \
                                                        trans_cat_test['movieId'])) \
                                     , shape=(n_users, n_items))
    return train, test, train_df, ratings

In [151]:
train, test, raw_train_df, ratings = informed_train_test(rating_df, .8)

C:\Users\87960\AppData\Local\Temp\ipykernel_16484\1912865.py:2: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  split_cut = np.int(np.round(rating_df.shape[0] * train_ratio))


In [145]:
start_time = time.time()
model=LightFM(no_components=115,learning_rate=0.027,loss='warp')
model.fit(train,epochs=12,num_threads=4)
# with open('saved_model','wb') as f:
#     saved_model={'model':model}
#     pickle.dump(saved_model, f)
auc_train = auc_score(model, train).mean()
auc_test = auc_score(model, test).mean()

print("--- Run time:  {} mins ---".format((time.time() - start_time)/60))
print("Train AUC Score: {}".format(auc_train))
print("Test AUC Score: {}".format(auc_test))

--- Run time:  0.001303410530090332 mins ---
Train AUC Score: 0.9854506850242615
Test AUC Score: 0.40725937485694885
